In [1]:
%load_ext autoreload
%autoreload 2

---
# Project 2 - Part 1
For the first part, use [the pretrained Bert model from hugging face](https://huggingface.co/transformers/task_summary.html#question-answering) and feed it with the five 300 words long sections from the book of your choice.
These sections should be selected so they are: introducing the protagonist(s), the antagonist, the crime and crime scene, any significant evidence, and the resolution of the crime/a narrative that presents the case against the perpetrator.
The questions you should ask are about the identity and characteristics of the protagonist, antagonist/perpetrator, the nature and the setting of the crime or crime scene, the evidence, and the case against the perpetrator.
Document the questions, ask the questions and document the specificity and accuracy of the results.
Then, feed the whole novel as a context, and repeat the questions. Document the results

In [2]:
import spacy
from transformers import pipeline
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering
import tensorflow as tf
import os


2022-10-24 20:06:27.800063: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-24 20:06:27.863257: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-24 20:06:27.879434: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-24 20:06:28.158598: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [3]:
nlp = spacy.load("en_core_web_sm")
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = TFAutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
generator = pipeline(task="question-answering", model=model, tokenizer=tokenizer)

2022-10-24 20:06:29.583846: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-24 20:06:29.584818: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-24 20:06:29.584913: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-24 20:06:29.584956: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA

In [4]:
def Q_A (questions, text):
    for question in questions:
        inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="tf")
        input_ids = inputs["input_ids"].numpy()[0]

        outputs = model(inputs)
        answer_start_scores = outputs.start_logits
        answer_end_scores = outputs.end_logits

    # Get the most likely beginning of answer with the argmax of the score
        answer_start = tf.argmax(answer_start_scores, axis=1).numpy()[0]
    # Get the most likely end of answer with the argmax of the score
        answer_end = tf.argmax(answer_end_scores, axis=1).numpy()[0] + 1

        answer = tokenizer.convert_tokens_to_string(
            tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
        )

        print(f"Question: {question}")
        print(f"Answer: {answer}")

In [5]:
def split_text(text):
    filename = os.path.join('text',text)
    file = open(filename,encoding="utf8")
    word_list = file.readlines()
    word_list = str(word_list).split()
    for i in range(0, len(word_list), 300):
        yield word_list[i:i + 300]

In [6]:
questions = [
    "Who is the protagonist?",
    "Who is the perpetrator?",
    "Where was the first victim murdered?",
]

filename = os.path.join('text','antagonist.txt')
file = open(filename,encoding="utf8")
text = file.read()
Q_A (questions, text)


Question: Who is the protagonist?
Answer: [SEP]
Question: Who is the perpetrator?
Answer: [CLS] who is the perpetrator? [SEP] “ excellent! ” said stapleton. “ you are perfectly right to be wary and discreet. i am justly reproved for what i feel was an unjustifiable intrusion, and i promise you that i will not mention the matter again. ” we had come to a point where a narrow grassy path struck off from the road and wound away across the moor. a steep, boulder - sprinkled hill lay upon the right which had in bygone days been cut into a granite quarry. the face which was turned towards us formed a dark cliff, with ferns and brambles growing in its niches. from over a distant rise there floated a grey plume of smoke. “ a moderate walk along this moor - path brings us to merripit house, ” said he. “ perhaps you will spare an hour that i may have the pleasure of introducing you to my sister. ” my first thought was that i should be by sir henry ’ s side. but then i remembered the pile of pape

In [7]:
questions = [
    "Who is the protagonist?",
    "Who is the perpetrator?",
    "Where was the first victim murdered?",
]

filename = os.path.join('text','crimescene.txt')
file = open(filename,encoding="utf8")
text = file.read()
Q_A (questions, text)

Question: Who is the protagonist?
Answer: sir charles baskerville
Question: Who is the perpetrator?
Answer: sir charles baskerville
Question: Where was the first victim murdered?
Answer: baskerville hall


In [8]:
questions = [
    "Who is the protagonist?",
    "Who is the perpetrator?",
    "Where was the first victim murdered?",
]

filename = os.path.join('text','evidence.txt')
file = open(filename,encoding="utf8")
text = file.read()
Q_A (questions, text)

Question: Who is the protagonist?
Answer: 
Question: Who is the perpetrator?
Answer: 
Question: Where was the first victim murdered?
Answer: [SEP]


In [9]:
questions = [
    "Who is the protagonist?",
    "Who is the perpetrator?",
    "Where was the first victim murdered?",
]

filename = os.path.join('text','protaganists.txt')
file = open(filename,encoding="utf8")
text = file.read()
Q_A (questions, text)

Question: Who is the protagonist?
Answer: mr. sherlock holmes
Question: Who is the perpetrator?
Answer: 
Question: Where was the first victim murdered?
Answer: [SEP]


In [10]:
questions = [
    "Who is the protagonist?",
    "Who is the perpetrator?",
    "Where was the first victim murdered?",
]

filename = os.path.join('text','resolution.txt')
file = open(filename,encoding="utf8")
text = file.read()
Q_A (questions, text)

Question: Who is the protagonist?
Answer: [SEP]
Question: Who is the perpetrator?
Answer: [CLS] who is the perpetrator? [SEP]
Question: Where was the first victim murdered?
Answer: [CLS]


In [11]:
questions = [
    "Who is the protagonist?",
    "Who is the perpetrator?",
    "Where was the first victim murdered?",
]
word_list = list(split_text('the-hound-of-the-baskervilles.txt'))

for i in range(len(word_list)):
    text = ' '.join([str(item) for item in word_list[i]])

    Q_A (questions, text)

Question: Who is the protagonist?
Answer: 
Question: Who is the perpetrator?
Answer: 
Question: Where was the first victim murdered?
Answer: 
Question: Who is the protagonist?
Answer: [CLS]
Question: Who is the perpetrator?
Answer: [CLS]
Question: Where was the first victim murdered?
Answer: [SEP]
Question: Who is the protagonist?
Answer: [CLS] who is the protagonist? [SEP]
Question: Who is the perpetrator?
Answer: [SEP]
Question: Where was the first victim murdered?
Answer: [SEP]
Question: Who is the protagonist?
Answer: [CLS] who is the protagonist? [SEP]
Question: Who is the perpetrator?
Answer: [CLS]
Question: Where was the first victim murdered?
Answer: [CLS]
Question: Who is the protagonist?
Answer: hugo baskerville
Question: Who is the perpetrator?
Answer: [SEP]
Question: Where was the first victim murdered?
Answer: [CLS]
Question: Who is the protagonist?
Answer: [SEP] thing great black beast shape like hound large hound mortal eye rest. look thing tear throat hugo baskerville
Q